In [16]:
dict = {1: [1, 2, 3], 2: ['a', 'b', 'c']}

In [20]:
for name, inputs in dict.items():
    if type(inputs) is int:
        print('before continue')
        continue
    
    print('=====')

=====
=====


In [23]:
a = dict.keys()

In [24]:
np.array(dict[a])

TypeError: unhashable type: 'dict_keys'